- Backend: FastAPI (Python) + NumPy/SciPy.

-  Data transport: JSON (for simplicity) or Apache Arrow (for HPC).

-   Visualization: Plotly (return interactive plots, easy) or VTK (convenient).

Required libraries :
- uvicorn plotly pydantic fastapi (conda/mamba)


## Directory structure 

- pde-solver-api/
    - main.py              # FastAPI server
    - schemas.py           # Pydantic models
    -  solvers/            # Numerical methods
         - heat_equation.py
    -  tests/              # Unit tests

## Типичная схема дя входных данных 

schemas.py

In [1]:
from pydantic import BaseModel
from typing import List

class HeatEquationInput(BaseModel):
    alpha: float           # Thermal diffusivity
    length: float          # Domain length (e.g., 1 meter)
    nx: int                # Spatial grid points (e.g., 100)
    nt: int                # Time steps (e.g., 500)
    dt: float              # Time step size
    initial_condition: List[float]  # u(x,0) as array
    boundary_conditions: dict       # E.g., {"left": 0.0, "right": 0.0}

Можно для простоты использовать библиотеку json.

Далее солвер (в solvers/heat_equation.py)

In [2]:
import numpy as np

def solve_heat_equation(alpha: float, length: float, nx: int, nt: int, dt: float, 
                        initial_condition: np.ndarray, bc: dict) -> np.ndarray:
    """
    Solves the 1D heat equation using finite differences.
    Returns: Solution matrix u(x,t) as NumPy array.
    """
    dx = length / (nx - 1)
    u = np.zeros((nx, nt))
    u[:, 0] = initial_condition  # Set initial condition

    # Finite difference scheme
    for t in range(1, nt):
        u[0, t] = bc["left"]      # Boundary condition (left)
        u[-1, t] = bc["right"]    # Boundary condition (right)
        for x in range(1, nx-1):
            u[x, t] = u[x, t-1] + alpha * dt / dx**2 * (
                u[x+1, t-1] - 2*u[x, t-1] + u[x-1, t-1]  # direct scheme
            )
    return u

Далее код для вызова. Прекрасен тем, что можно запускать на другой машине, например, на каком-нибудь вычислительном кластере (ведь мы уже взрослые).

Код в файле main.py (если запускать пряом из этой тетрадки - то строку from schemas ...  нужно опустить )

In [4]:
from fastapi import FastAPI
from fastapi.responses import JSONResponse
import numpy as np
#from schemas import HeatEquationInput # uncomment this line to run from terminal
from solvers.heat_equation import solve_heat_equation
import plotly.graph_objects as go
import base64

app = FastAPI(title="PDE Solver API")

@app.post("/solve/heat-equation/")
async def solve_heat_eq(params: HeatEquationInput): # asynchronous call to computing server
    # Convert input to NumPy
    u0 = np.array(params.initial_condition)
    
    # Solve PDE
    solution = solve_heat_equation(
        alpha=params.alpha,
        length=params.length,
        nx=params.nx,
        nt=params.nt,
        dt=params.dt,
        initial_condition=u0,
        bc=params.boundary_conditions
    )

    # Generate Plotly figure
    fig = go.Figure(data=[go.Surface(z=solution.T)])
    fig.update_layout(title="Heat Equation Solution")
    plot_html = fig.to_html(full_html=False)

    # Return solution + plot
    return {
        "solution": solution.tolist(),  # Convert to JSON-serializable
        "visualization": plot_html     # Embeddable Plotly HTML
    }

ModuleNotFoundError: No module named 'solvers'

 Запуск приложения на стороне сервера (если на вашем компе, то прямо у вас из командной строки).

```bash
uvicorn main:app --reload
```


Запрос на запуск солвера 

```bash
curl -X POST "http://localhost:8000/solve/heat-equation/" \
-H "Content-Type: application/json" \
-d '{
    "alpha": 0.01,
    "length": 1.0,
    "nx": 50,
    "nt": 100,
    "dt": 0.1,
    "initial_condition": [0, ... , 0],  # Array of length nx
    "boundary_conditions": {"left": 0.0, "right": 0.0}
}'

```

Ответ сервера в виде словаря

```bash
{
  "solution": [[0, 0, ...], ...],  # 2D array of u(x,t)
  "visualization": "<div>...</div>" # Plotly HTML
}
```